<a href="https://colab.research.google.com/github/slazur83/Tableau/blob/main/Sport%20Activity/get_steps_from_garmin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install garminconnect

In [5]:
from datetime import date, timedelta
from pathlib import Path
import pandas as pd
import os


def choose_start_date():
    """Ask the user and return a start date."""
    current_year = date.today().year

    print("Choose start date option:")
    print("1: All available data (starting date from: 2020-12-21)")
    print(f"2: From beginning of current year (starting date from: {current_year}-01-01)")
    print("3: Enter custom start date")

    choice = input("Enter 1, 2, or 3: ").strip()

    if choice == "1":
        return date(2020, 12, 21)
    elif choice == "2":
        return date(current_year, 1, 1)
    elif choice == "3":
        user_input = input("Enter custom start date (YYYY-MM-DD): ").strip()
        try:
            return date.fromisoformat(user_input)
        except ValueError:
            print(f"Invalid format. Using start of current year ({current_year}-01-01).")
            return date(current_year, 1, 1)
    else:
        print(f"Invalid choice. Using start of current year ({current_year}-01-01).")
        return date(current_year, 1, 1)


# Paths
TOKEN_DIR = "/content/drive/MyDrive/Programowanie/Google Colab/Tableau/Data/Garmin tokens"
IMPORT_DIR = "/content/drive/MyDrive/Programowanie/Google Colab/Tableau/Data/Garmin imports"
os.makedirs(IMPORT_DIR, exist_ok=True)


# Login
garmin = Garmin()
garmin.login(TOKEN_DIR)


# Verify login
try:
    garmin.get_user_profile()
except Exception as e:
    raise Exception(f"Login failed! Check your tokens. Details: {e}")

start = choose_start_date()
end = date.today()


# Fetch data
rows = []
current = start

while current <= end:
    iso = current.isoformat()
    try:
        summary = garmin.get_user_summary(iso)
        steps = summary.get("totalSteps") or summary.get("steps") or 0
    except Exception:
        steps = 0

    rows.append({"Date": iso, "Steps": steps})
    current += timedelta(days=1)

df = pd.DataFrame(rows)


# Save CSV
IMPORT_DIR = Path(IMPORT_DIR)
file_name = f"steps_export_{start.isoformat()}_to_{end.isoformat()}.csv"
file_path = IMPORT_DIR / file_name

df.to_csv(file_path, index=False)
print(f"File saved to: {file_path}")


Choose start date option:
1: All available data (starting 2020-12-21)
2: From beginning of current year (2025-01-01)
3: Enter custom start date
Enter 1, 2, or 3: 2
File saved to /content/drive/MyDrive/Programowanie/Google Colab/Tableau/Data/Garmin imports/steps_export_2025-01-01_to_2025-12-01.csv
